In [2]:
import pandas as pd
import numpy as np
import openpyxl
import io
import requests

In [3]:
url = 'https://raw.githubusercontent.com/amitsinghh1/combo-calculation/main/mamaearth.csv'
download = requests.get(url).content

In [4]:
df = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [5]:
df.head()

,Display Order Code,abc,Item SKU Code,Sale Order Status,Sale Order Item Status,Shipping Package Status Code,Facility,Created,Channel Shipping,Item Details
0,91013155713,8.904420e+12,1,PROCESSING,FULFILLABLE,CREATED,SRF Pataudi,13-02-2023 09:41,False,NaN
1,91013155713,8.904420e+12,1,PROCESSING,FULFILLABLE,CREATED,SRF Pataudi,13-02-2023 09:41,False,NaN
2,91013155713,8.904420e+12,1,PROCESSING,FULFILLABLE,CREATED,SRF Pataudi,13-02-2023 09:41,False,NaN
3,91013155713,8.904420e+12,1,PROCESSING,FULFILLABLE,CREATED,SRF Pataudi,13-02-2023 09:41,False,NaN
4,91013155713,8.904420e+12,1,PROCESSING,FULFILLABLE,CREATED,SRF Pataudi,13-02-2023 09:41,False,NaN


In [6]:
df.rename(columns={"Display Order Code":"order_id","Item SKU Code":"sku"},inplace=True)
df['sku']=df['sku'].astype('category')
label1 = dict(enumerate(df['sku'].cat.categories))
df['sku']=df['sku'].astype('category').cat.codes
df.sort_values(by=["order_id","sku"],inplace=True)

In [7]:
#combo data vs frequency
df_a=df[['order_id','sku']].groupby('order_id').size().reset_index(name="qty")
df_a['combo']=df[['order_id','sku']].groupby('order_id')['sku'].apply(list).reset_index(name='combo')['combo']
df_a['combo_dist_tuple']=df[['order_id','sku']].groupby('order_id')['sku'].unique().apply(tuple).reset_index(name='combo_dt')['combo_dt']
tuple_qty=[]
for i in df_a['combo_dist_tuple']:
    len_tuple=len(i)
    tuple_qty.append(len_tuple)
df_a['len_tuple']=tuple_qty   

In [8]:
df_a.head()

,order_id,qty,combo,combo_dist_tuple,len_tuple
0,91012941101-1,5,"[88, 165, 224, 314, 315]","(88, 165, 224, 314, 315)",5
1,91013082948-1,2,"[190, 190]","(190,)",1
2,91013087664-1,5,"[41, 41, 41, 197, 197]","(41, 197)",2
3,91013097130-1,1,[75],"(75,)",1
4,91013099204-1,3,"[88, 314, 315]","(88, 314, 315)",3


In [9]:
df_a.qty.mean()

4.548937146352653

In [10]:
df_b=df_a.groupby('combo_dist_tuple').size().reset_index(name='freq').sort_values('freq',ascending=False)
df_b['order_list']=df_a.groupby('combo_dist_tuple')['order_id'].apply(list).reset_index(name='a')['a']
df_b.head(10)

,combo_dist_tuple,freq,order_list
4590,"(41,)",181,"[91013111244-1, 91013205771, 91013206562, 9101..."
237,"(1, 2)",123,"[91013162941, 91013163009, 91013163584, 910131..."
4,"(0, 1, 2, 3, 4)",87,"[91013155713, 91013181343, 91013187715, 910131..."
4640,"(41, 76)",83,"[91013182655, 91013208616, 91013218518, 910132..."
6061,"(76,)",81,"[91013254765, 91013255727, 91013259101, 910132..."
971,"(5, 7)",68,"[91013161575, 91013165099, 91013171170, 910131..."
4364,"(40,)",67,"[91013253718, 91013259399, 91013262813, 910132..."
22,"(0, 1, 2, 3, 4, 45)",65,"[91013159638, 91013173283, 91013173931, 910132..."
2831,"(24,)",64,"[91013257951, 91013258246, 91013258463, 910132..."
5494,"(53,)",59,"[91013203489, 91013212343, 91013213973, 910132..."


In [11]:
min_freq=25 ##select min frequency which you want
result_combo=df_b[df_b['freq']>=min_freq]  

In [12]:
result_combo

,combo_dist_tuple,freq,order_list
4590,"(41,)",181,"[91013111244-1, 91013205771, 91013206562, 9101..."
237,"(1, 2)",123,"[91013162941, 91013163009, 91013163584, 910131..."
4,"(0, 1, 2, 3, 4)",87,"[91013155713, 91013181343, 91013187715, 910131..."
4640,"(41, 76)",83,"[91013182655, 91013208616, 91013218518, 910132..."
6061,"(76,)",81,"[91013254765, 91013255727, 91013259101, 910132..."
971,"(5, 7)",68,"[91013161575, 91013165099, 91013171170, 910131..."
4364,"(40,)",67,"[91013253718, 91013259399, 91013262813, 910132..."
22,"(0, 1, 2, 3, 4, 45)",65,"[91013159638, 91013173283, 91013173931, 910132..."
2831,"(24,)",64,"[91013257951, 91013258246, 91013258463, 910132..."
5494,"(53,)",59,"[91013203489, 91013212343, 91013213973, 910132..."


In [13]:
result_combo.freq.sum()

2033

In [18]:
result1=pd.DataFrame(columns=['OrderID','Tracking','code','batch','forceupdate','Channel_code'])
k=0
l=1
for i in result_combo['order_list']:
  for j in i:
    result1.at[k,'OrderID']=j
    result1.at[k,'Channel_code']='Mamaearth_Production'
    result1.at[k,'batch']=l
    k=k+1
  l=l+1
   
result1.to_excel(r"combo_result.xlsx")

In [19]:
#now finding max order containing SKU pool

#preparing rest of the data after removing combo orders ids

df_rest=df_b[df_b['freq']<min_freq]

rest_orders=[]
for i in df_rest.order_list:
    rest_orders.extend(i)


df_rest=df[df['order_id'].isin(rest_orders)]
df_a=df_a[df_a['order_id'].isin(rest_orders)]

In [20]:
df_rest.head()

,order_id,abc,sku,Sale Order Status,Sale Order Item Status,Shipping Package Status Code,Facility,Created,Channel Shipping,Item Details
26040,91012941101-1,8.906090e+12,88,PROCESSING,FULFILLABLE,CREATED,SRF Pataudi,14-02-2023 16:35,False,NaN
26042,91012941101-1,8.906090e+12,165,PROCESSING,FULFILLABLE,CREATED,SRF Pataudi,14-02-2023 16:35,False,NaN
26043,91012941101-1,8.906090e+12,224,PROCESSING,FULFILLABLE,CREATED,SRF Pataudi,14-02-2023 16:35,False,NaN
26039,91012941101-1,8.906090e+12,314,PROCESSING,FULFILLABLE,CREATED,SRF Pataudi,14-02-2023 16:35,False,NaN
26041,91012941101-1,8.906090e+12,315,PROCESSING,FULFILLABLE,CREATED,SRF Pataudi,14-02-2023 16:35,False,NaN


In [21]:
######## Below code is to find SKUs pool #########

In [22]:
###select number of SKUs out of which combos will be created,these are most repetetive SKUs.

n=25   ##POOL STRENGTH

df_c=df_rest[['sku']].groupby('sku').size().reset_index(name='count').sort_values(by=['count'], ascending=False)
topn=df_c.sort_values(by=['count'], ascending=False).head(n)['sku'].astype(np.int64).to_list()
topn.sort()

################################ Combo formation ###################################

combo_size=3

if combo_size==3:    
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                combo[0]=topn[i]
                combo[1]=topn[j]
                combo[2]=topn[k]
                a.append(combo.copy())

elif combo_size==4:    
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    combo[0]=topn[i]
                    combo[1]=topn[j]
                    combo[2]=topn[k]
                    combo[3]=topn[l]
                    a.append(combo.copy())
                   
elif combo_size==5:
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    for m in range(l+1,len(topn)):
                        combo[0]=topn[i]
                        combo[1]=topn[j]
                        combo[2]=topn[k]
                        combo[3]=topn[l]
                        combo[4]=topn[m]
                        a.append(combo.copy())
    

In [23]:
len(a)

2300

In [24]:
result_df=pd.DataFrame(columns=['combo',"freq"])
result_df['combo']=a


In [25]:
## Searching number of fully inclusive orders in a given pair of SKUs

df3=df_rest[['order_id','sku']].groupby('order_id').size().reset_index(name="qty")
df3['sku_list']=df_rest[['order_id','sku']].groupby('order_id')['sku'].unique().apply(list).reset_index(name='combo')['combo']

length_orders=df3['order_id'].count() 

res_list=[]
for sample in result_df['combo']:
    mfg=0
    for j in range(length_orders):
        test_list=df3.sku_list[j]
        check=all(item in sample for item in test_list) 
        if check is True:
            mfg=mfg+1
        else:
            mfg=mfg
    res_list.append(mfg)
        
result_df['freq']=res_list
         


In [26]:
result_df.sort_values("freq",ascending=False).head(8)

,combo,freq
45,"[0, 3, 4]",105
2,"[0, 1, 4]",87
67,"[0, 4, 7]",80
71,"[0, 4, 38]",79
24,"[0, 2, 4]",76
298,"[1, 3, 4]",72
324,"[1, 4, 38]",71
68,"[0, 4, 21]",71


In [27]:
combo=[0,3,4]

In [28]:
df_x=df_a[['order_id','combo']]
a=[]
for sample in df_x.combo:
    test=all(item in combo for item in sample)
    if test is True:
        a.append('yes')
    else:
        a.append('no')
df_x['order_in_pool']=a 

<ipython-input-28-6e200762e202>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['order_in_pool']=a


In [29]:
df_x[df_x['order_in_pool']=='yes'].count()


order_id         105
combo            105
order_in_pool    105
dtype: int64

In [30]:
df_x[df_x['order_in_pool']=='yes'].to_excel(r'rest.xlsx')

In [31]:
df_med=df_x[df_x['order_in_pool']=='no']
exclude_list=df_med['order_id'].to_list()

df_rest=df[df['order_id'].isin(exclude_list)]
df_a=df_a[df_a['order_id'].isin(exclude_list)]

In [46]:
df_med=df_x[df_x['order_in_pool']=='no']
exclude_list=df_med['order_id'].to_list()

df_rest=df[df['order_id'].isin(exclude_list)]
df_a=df_a[df_a['order_id'].isin(exclude_list)]

n=25 ##POOL STRENGTH
df_c=df_rest[['sku']].groupby('sku').size().reset_index(name='count').sort_values(by=['count'], ascending=False)
topn=df_c.sort_values(by=['count'], ascending=False).head(n)['sku'].astype(np.int64).to_list()
topn.sort()

################################ Combo formation ###################################

combo_size=3

if combo_size==2:    
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
                combo[0]=topn[i]
                combo[1]=topn[j]
                a.append(combo.copy())
                         

elif combo_size==3:    
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                combo[0]=topn[i]
                combo[1]=topn[j]
                combo[2]=topn[k]
                a.append(combo.copy())

elif combo_size==4:    
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    combo[0]=topn[i]
                    combo[1]=topn[j]
                    combo[2]=topn[k]
                    combo[3]=topn[l]
                    a.append(combo.copy())
                   
elif combo_size==5:
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    for m in range(l+1,len(topn)):
                        combo[0]=topn[i]
                        combo[1]=topn[j]
                        combo[2]=topn[k]
                        combo[3]=topn[l]
                        combo[4]=topn[m]
                        a.append(combo.copy())
elif combo_size==6:
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    for m in range(l+1,len(topn)):
                        for n in range(m+1,len(topn)):
                            combo[0]=topn[i]
                            combo[1]=topn[j]
                            combo[2]=topn[k]
                            combo[3]=topn[l]
                            combo[4]=topn[m]
                            combo[5]=topn[n]
                            a.append(combo.copy())
elif combo_size==8:
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    for m in range(l+1,len(topn)):
                        for n in range(m+1,len(topn)):
                            for o in range(n+1,len(topn)):
                                for p in range(o+1,len(topn)):
                                    combo[0]=topn[i]
                                    combo[1]=topn[j]
                                    combo[2]=topn[k]
                                    combo[3]=topn[l]
                                    combo[4]=topn[m]
                                    combo[5]=topn[n]
                                    combo[6]=topn[o]
                                    combo[7]=topn[p]
                                    a.append(combo.copy()) 

result_df1=pd.DataFrame(columns=['combo',"freq"])
result_df1['combo']=a
##------------------------------------------------------------------------------------------
df3=df_rest[['order_id','sku']].groupby('order_id').size().reset_index(name="qty")
df3['sku_list']=df_rest[['order_id','sku']].groupby('order_id')['sku'].unique().apply(list).reset_index(name='combo')['combo']

##-------------------------------------------------------------------------------------------
length_orders=df3['order_id'].count() 

res_list=[]
for sample in result_df1['combo']:
    mfg=0
    for j in range(length_orders):
        test_list=df3.sku_list[j]
        check=all(item in sample for item in test_list) 
        if check is True:
            mfg=mfg+1
        else:
            mfg=mfg
    res_list.append(mfg)
        
result_df1['freq']=res_list
result_df1.sort_values("freq",ascending=False).head(8)

,combo,freq
3601,"[42, 110, 122]",59
2768,"[24, 40, 42]",58
4052,"[88, 110, 122]",58
2516,"[18, 110, 122]",57
615,"[1, 38, 42]",55
2411,"[18, 42, 110]",55
780,"[1, 110, 122]",55
3595,"[42, 88, 110]",54


In [32]:
combo=[42,110,122]

In [33]:
df_x=df_a[['order_id','combo']]
a=[]
for sample in df_x.combo:
    test=all(item in combo for item in sample)
    if test is True:
        a.append('yes')
    else:
        a.append('no')
df_x['order_in_pool']=a

<ipython-input-33-3a8872deaf4c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['order_in_pool']=a


In [37]:
df_x[df_x['order_in_pool']=='yes'].to_excel(r'rest1.xlsx')

BadZipFile: ignored

In [ ]:
df_med=df_x[df_x['order_in_pool']=='no']
exclude_list=df_med['order_id'].to_list()

df_rest=df[df['order_id'].isin(exclude_list)]
df_a=df_a[df_a['order_id'].isin(exclude_list)]

In [ ]:
df_a

,order_id,qty,combo,combo_dist_tuple,len_tuple
1,1842704,2,"[16, 110]","(16, 110)",2
2,1855474,3,"[47, 49, 117]","(47, 49, 117)",3
3,1855562,5,"[41, 41, 57, 60, 117]","(41, 57, 60, 117)",4
4,1855606,5,"[41, 47, 51, 84, 117]","(41, 47, 51, 84, 117)",5
5,1855613,3,"[41, 82, 117]","(41, 82, 117)",3
...,...,...,...,...,...
1889,SO52493,1,[38],"(38,)",1
1890,SO52494,1,[34],"(34,)",1
1891,SO52495,1,[32],"(32,)",1
1892,SO52496,1,[65],"(65,)",1
